# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np 
import pandas as pd


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Load the data from [here](https://drive.google.com/file/d/1AmdvTSj5Fl86i4uHiOnZPFg8S8mAIreA/view?usp=sharing) and place it in the data folder

In [2]:
# Your code here:
df = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [7]:
# Your code here:
df[['Annual Salary']].head(10)

,Annual Salary
0,101442.0
1,94122.0
2,101592.0
3,110064.0
4,NaN
5,50436.0
6,NaN
7,NaN
8,103350.0
9,93354.0


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [16]:
# Your code here:
df.isna().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [26]:
# Your code here:
df['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [30]:
# Your code here:
df.groupby('Department')['Name'].count()


Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [53]:
from scipy import stats
from scipy.stats import ttest_1samp
# Your code here:
p = 0.05
mean = 30
if stats.ttest_1samp(df.loc[df['Salary or Hourly'] == 'Hourly']['Hourly Rate'], popmean=mean)[1] < p:
    print('We reject that the mean of hourly wage of all hourly workers is  ${0:.0f}/hr.'.format(mean))
else:
    print('We cannot reject that the mean of hourly wage of all hourly workers is  ${0:.0f}/hr.'.format(mean))


We reject that the mean of hourly wage of all hourly workers is  $30/hr.


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [68]:
# Your code here:
df.loc[df.Department == 'POLICE']['Annual Salary']
p = 0.05
mean = 86000
if stats.ttest_1samp(df.loc[(df.Department == 'POLICE') & (df['Salary or Hourly'] == 'Salary')]['Annual Salary'], popmean=mean)[1]/2 < p:
    print('We reject that the mean of yearly wage of all police personal  ${0:.0f}/hr.'.format(mean))
else:
    print('We reject that the mean of yearly wage of all police personal  ${0:.0f}/hr.'.format(mean))

We reject that the mean of yearly wage of all police personal  $86000/hr.


Using the `crosstab` function, find the department that has the most hourly workers. 

In [106]:
# Your code here:
print('The deparment {} has the most Hourly workers'.format(
      pd.crosstab(df['Department'],df['Salary or Hourly']).sort_values('Hourly',ascending=False).reset_index()['Department'][0])
     )

The deparment STREETS & SAN has the most Hourly workers


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [122]:
# Your code here:

def one_side_test(alpha,mean,array,test_type):
    
    t, p = stats.ttest_1samp(array, mean)
    
    print('t: {0:.5f} and p: {1:.10f}'.format(t,p))
    
    if test_type == 'higher' and (p/2 < alpha and t > 0) :
        print('Reject null hypothesis that mean =  {}'.format(mean))
    if test_type == 'lower' and (p/2 < alpha and t < 0) :
        print('Reject null hypothesis that mean =  {}'.format(mean))
        
p = 0.05
mean = 35

one_side_test(alpha = p,
              mean = mean,
              array = df.loc[(df.Department == 'STREETS & SAN') & (df['Salary or Hourly'] == 'Hourly')]['Hourly Rate'],
              test_type = 'lower'
             )


t: -9.56745 and p: 0.0000000000
Reject null hypothesis that mean =  35


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [128]:
df_hourly.head(2)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10


In [139]:
# Your code here:
df_hourly = df.loc[df['Salary or Hourly'] == 'Hourly']
dof = len(df_hourly) - 1 

t = stats.t.interval(0.95,dof)[1]
x  = df_hourly['Hourly Rate'].mean()
se = np.std(df_hourly['Hourly Rate']) / np.sqrt(len(df_hourly))

con_int = [round(num,3) for num in [x - (t * se) , x + (t * se)]]
print('With 95 % confidence, the average salary of hourly workers lies in the inverval:', con_int)


With 95 % confidence, the average salary of hourly workers lies in the inverval: [32.523, 33.054]


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [141]:
# Your code here:

df_police = df.loc[(df['Salary or Hourly'] == 'Salary') & (df['Department'] == 'POLICE')]
dof = len(df_police) - 1 

t = stats.t.interval(0.95,dof)[1]
x  = df_police['Annual Salary'].mean()
se = np.std(df_police['Annual Salary']) / np.sqrt(len(df_police))

con_int = [round(num,3) for num in [x - (t * se) , x + (t * se)]]
print('With 95 % confidence, the average salary of hourly workers lies in the inverval:', con_int)


With 95 % confidence, the average salary of hourly workers lies in the inverval: [86177.068, 86795.761]


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

